# Simulation

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import facets_drivers

%store -r notebook_data
globals().update(notebook_data)

In [ ]:
if not local_run:
    from ewoksdask.schedulers import slurm_scheduler
    cluster = slurm_scheduler(account="staff-id01",queue="nice", walltime="01:00:00", memory="40GB", cores=num_cores)

In [ ]:
from ewoksdask import execute_graph

nodes = [
    {
        "id": "get_orientation",
        "task_type": "class",
        "task_identifier": "facets_drivers.theory.GetOrientation",
    },
    {
        "id": "nanoresculpted",
        "task_type": "class",
        "task_identifier": "facets_drivers.simulation.RunNanoReSculpted",
    },
    {
        "id": "lammps",
        "task_type": "class",
        "task_identifier": "facets_drivers.simulation.RunLammps",
    },
]

links = [
    {
        "source": "get_orientation",
        "target": "nanoresculpted",
        "data_mapping": [{"source_output": "nsparam_filepath", "target_input": "nsparam_filepath"}],
    },
    {
        "source": "nanoresculpted",
        "target": "lammps",
        "data_mapping": [{"source_output": "pos_filepath", "target_input": "pos_filepath"}],
    },
]

workflow = {"graph": {"id": "simulation"}, "nodes": nodes, "links": links}

positive_hkl = ",".join(str(i) for i in hkl)
negative_hkl = ",".join(str(-i) for i in hkl)
exclude_hkl = f"{positive_hkl};{negative_hkl}"
inputs = [
    {"id": "get_orientation", "name": "scratch_directory", "value": simulation_directory},
    {"id": "get_orientation", "name": "vtp_facets_filepath", "value": f"{theory_directory}/exp_1.vtp"},
    {"id": "get_orientation", "name": "hkl", "value": "".join(str(i) for i in hkl)},
    {"id": "get_orientation", "name": "exclude_hkl", "value": exclude_hkl},
    
    {"id": "nanoresculpted", "name": "scratch_directory", "value": simulation_directory},
    {"id": "nanoresculpted", "name": "obj_filepath", "value": f"{theory_directory}/exp.obj"},
    {"id": "nanoresculpted", "name": "num_procs", "value": f"{num_cores}"},

    {"id": "lammps", "name": "scratch_directory", "value": simulation_directory},
    {"id": "lammps", "name": "num_procs", "value": f"{num_cores}"},
]

In [ ]:
%%time
result = execute_graph(workflow, inputs=inputs, scheduler=None if local_run else cluster.scheduler_address, outputs=[
    {'id': 'nanoresculpted'}, 
    {'id': 'lammps'}
], merge_outputs=False)

In [ ]:
[print(result[key]["result"]["full_log"]) for key in result]

In [ ]:
if not local_run:
    cluster.close()